<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/2way_ANOVA_within_Tukey_Kramer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas statsmodels

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import MultiComparison

# CSVファイルのパス
csv_file_path = '/content/result_exp1_within.csv'

# CSVファイルの読み込み
data = pd.read_csv(csv_file_path)

# データの確認
print("データの先頭部分:\n", data.head())

# 因子Aと因子Bと被験者IDの名前を適切に設定してください
factorA = 'factorA'  # 被験者内要因Aを示す列名
factorB = 'factorB'  # 被験者内要因Bを示す列名
subject_column = 'subject'  # 被験者IDを示す列名
value_column = 'value'  # 値を示す列名

# 被験者内要因に対する2要因分散分析の実行
anova_rm = AnovaRM(data, depvar=value_column, subject=subject_column, within=[factorA, factorB])
anova_results = anova_rm.fit()
anova_table = anova_results.anova_table

# 分散分析結果の表示
print("\n被験者内要因に対する2要因分散分析の結果:\n", anova_table)

# 誤差平方和の計算
anova_table['Sum Sq'] = anova_table['F Value'] * anova_table['Num DF'] * anova_table['Den DF']

# 誤差平方和の合計を計算し、残差の平方和として使用
total_sum_sq = data[value_column].var() * (len(data) - 1)  # 総平方和
residual_sum_sq = total_sum_sq - anova_table['Sum Sq'].sum()  # 残差平方和

# 部分η²の計算
anova_table['Partial η²'] = anova_table['Sum Sq'] / (anova_table['Sum Sq'] + residual_sum_sq)
print("\n部分η²を含む分散分析結果:\n", anova_table)

# 因子Aに対するTukey-Kramer HSDの実行
mc_A = MultiComparison(data[value_column], data[factorA])
result_A = mc_A.tukeyhsd()
print("\nTukey-Kramer HSDの結果 (因子Aの比較):")
print(result_A)
print(result_A.summary())

# 因子Bに対するTukey-Kramer HSDの実行
mc_B = MultiComparison(data[value_column], data[factorB])
result_B = mc_B.tukeyhsd()
print("\nTukey-Kramer HSDの結果 (因子Bの比較):")
print(result_B)
print(result_B.summary())

# 交互作用効果に対するTukey-Kramer HSD検定のための新しい列を作成
data['interaction'] = data[factorA].astype(str) + ':' + data[factorB].astype(str)
mc_interaction = MultiComparison(data[value_column], data['interaction'])
result_interaction = mc_interaction.tukeyhsd()
print("\nTukey-Kramer HSDの結果 (交互作用の比較):")
print(result_interaction)
print(result_interaction.summary())

データの先頭部分:
    subject       value factorA factorB
0        1  326.616628  moring     pre
1        2  237.014272  moring     pre
2        3  180.624471  moring     pre
3        4  403.583413  moring     pre
4        5  186.317823  moring     pre

被験者内要因に対する2要因分散分析の結果:
                    F Value  Num DF  Den DF    Pr > F
factorA          12.458347     1.0    20.0  0.002105
factorB           7.568219     1.0    20.0  0.012318
factorA:factorB   2.922103     1.0    20.0  0.102846

部分η²を含む分散分析結果:
                    F Value  Num DF  Den DF    Pr > F      Sum Sq  Partial η²
factorA          12.458347     1.0    20.0  0.002105  249.166938    0.000453
factorB           7.568219     1.0    20.0  0.012318  151.364385    0.000275
factorA:factorB   2.922103     1.0    20.0  0.102846   58.442068    0.000106

Tukey-Kramer HSDの結果 (因子Aの比較):
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1 group2 meandiff p-adj   lower   upper  reject
-----------------------------------------------------
e